In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_FilmTrust'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/filmtrust'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [4]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side.pkl','rb'))

In [5]:
trainusers=set(dftrain['userId'].unique())
validusers=set(dfvalid['userId'].unique())
trainitems=set(dftrain['itemId'].unique())
validitems=set(dfvalid['itemId'].unique())

In [6]:
commonusers=trainusers.intersection(validusers)
commonitems=trainitems.intersection(validitems)

In [7]:
dfvalid['cold_start_user']=~dfvalid['userId'].isin(commonusers)
dfvalid['cold_start_item']=~dfvalid['itemId'].isin(commonitems)

In [8]:
print (pd.crosstab(dfvalid['cold_start_user'],dfvalid['cold_start_item']))
print (pd.crosstab(dfvalid['cold_start_user'],dfvalid['cold_start_item'])/dfvalid.shape[0])   

cold_start_item  False  True 
cold_start_user              
False             3109     56
True               278     15
cold_start_item     False     True 
cold_start_user                    
False            0.899075  0.016194
True             0.080393  0.004338


In [9]:
dftrain['userId'].value_counts()

272     224
1187    218
3       193
161     189
79      172
199     155
969     154
702     134
670     129
845     114
1039    106
1333    105
188     102
323     101
1199     98
1249     96
355      92
842      91
830      87
1034     84
1212     84
308      84
316      79
626      79
1153     79
1147     77
1157     75
533      73
591      73
1386     70
       ... 
268       1
113       1
855       1
252       1
558       1
1058      1
1477      1
904       1
809       1
1275      1
54        1
1136      1
1323      1
1421      1
1411      1
1264      1
230       1
1362      1
76        1
1181      1
582       1
1171      1
1107      1
934       1
140       1
1456      1
941       1
1490      1
247       1
1322      1
Name: userId, Length: 1481, dtype: int64

In [10]:
dfvalid['cold_start_group']=np.where(dfvalid['cold_start_user'] & dfvalid['cold_start_item'],'User-Item-cold-start',
                                    np.where((~dfvalid['cold_start_user']) & (~dfvalid['cold_start_item']), 'No-cold-start',\
                                            np.where(dfvalid['cold_start_user'], 'User-Cold-Start', 'Item-Cold_start')))    


In [11]:
dfvalid['cold_start_bucket']=np.where(dfvalid['cold_start_group']=='No-cold-start',0,1)

In [12]:
dfvalid['cold_start_group'].value_counts()

No-cold-start           3109
User-Cold-Start          278
Item-Cold_start           56
User-Item-cold-start      15
Name: cold_start_group, dtype: int64

In [13]:
dfvalid['cold_start_bucket'].value_counts()

0    3109
1     349
Name: cold_start_bucket, dtype: int64

In [14]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df_side_cold.pkl','wb'))